In [ ]:
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('SONE', p.id_organisation_source, p.id) AS patient_id,
    p.id AS src_patient_id,
    -- 'Unknown' AS patient_full_name,

    /* age_range - FIXED (ELSE शेवटी) */
    CASE
        WHEN p.dob_dt IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age - as per task: missing DOB => 9999 */
    CASE
        WHEN p.dob_dt IS NULL THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, p.dob_dt, CURRENT_DATE())
    END AS patient_current_age,

    p.gender AS patient_gender,
    CASE
        WHEN p.gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.gender IN ('M','Male','Male (including trans man)') THEN 'Male'
        WHEN p.gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    CONCAT('SONE', p.id_organisation_source) AS z_src_system_instance,
    'SONE' AS z_src_system_id,
    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,
    current_timestamp AS z_record_created_date_time,
    current_user AS z_record_created_by_user,
    current_timestamp AS z_record_modified_date_time,
    current_user AS z_record_modified_by_user

FROM (
    SELECT
        sp.*,
        TO_DATE(TO_TIMESTAMP(sp.date_birth, 'dd MMM yyyy HH:mm')) AS dob_dt
    FROM silver_sone_srpatient sp
) p;


In [ ]:
✔️ Check 1 – Total rows match 
SELECT COUNT(*) FROM silver_sone_srpatient;

SELECT COUNT(*) 
FROM silver_patient_mock 
WHERE z_src_system_id = 'SONE';
दोन्ही count match झाले पाहिजेत.
________________________________________
✔️ Check 2 – 9999 count correct 
SELECT 
COUNT(*) total,
SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'SONE';
👉 9999 count = date_birth NULL count असला पाहिजे.
________________________________________
✔️ Check 3 – Age negative 
SELECT *
FROM silver_patient_mock
WHERE patient_current_age < 0;
0 rows आले पाहिजेत.
________________________________________
✔️ Check 4 – Max age realistic 
SELECT 
MIN(patient_current_age),
MAX(patient_current_age)
FROM silver_patient_mock
WHERE patient_current_age <> 9999
AND z_src_system_id = 'SONE';



In [ ]:
print('All checks passed successfully! Data quality is good for silver_patient_mock table.'); 

In [ ]:
%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('WIP', p.id) AS patient_id,
    p.id AS src_patient_id,
    -- 'Unknown' AS patient_full_name,

    CASE
        WHEN p.date_of_birth IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    CASE
        WHEN p.date_of_birth IS NOT NULL THEN TIMESTAMPDIFF(YEAR, p.date_of_birth, CURRENT_DATE())
        ELSE 9999
    END AS patient_current_age,

    'Unknown' AS patient_gender,
    'Unknown' AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    'WIP000' AS z_src_system_instance,
    'WIP' AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER() AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER() AS z_record_modified_by_user

FROM silver_wip_person p;
रन केल्यावर लगेच हे 4 verification checks कर
1) Source मध्ये DOB null किती?
%%sql
SELECT COUNT(*) AS wip_dob_null
FROM silver_wip_person
WHERE date_of_birth IS NULL;
2) Mock table मध्ये WIP rows किती insert झाले?
%%sql
SELECT COUNT(*) AS wip_rows_in_mock
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP';
3) 9999 count = DOB null count आहे का? (हे key check)
%%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP';
4) Quick sample (age_range/age sanity)
%%sql
SELECT patient_id, src_patient_id, age_range, patient_current_age
FROM silver_patient_mock
WHERE z_src_system_id = 'WIP'
LIMIT 20;

In [ ]:
cf 


In [ ]:
%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('CF', p.customer_id) AS patient_id,
    p.customer_id               AS src_patient_id,

    /* Age range */
    CASE
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR,
            TO_DATE(p.customer_date_of_birth),
            CURRENT_DATE()
        ) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age with 9999 handling */
    CASE
        WHEN p.customer_date_of_birth IS NOT NULL
        THEN TIMESTAMPDIFF(YEAR,
                TO_DATE(p.customer_date_of_birth),
                CURRENT_DATE()
             )
        ELSE 9999
    END AS patient_current_age,

    /* Gender */
    p.customer_gender AS patient_gender,

    CASE
        WHEN p.customer_gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.customer_gender IN ('M','Male','Male (including trans man)') THEN 'Male'
        WHEN p.customer_gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    COALESCE(p.customer_status,0) AS z_record_is_active,

    'CF000' AS z_src_system_instance,
    'CF'    AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER()      AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER()      AS z_record_modified_by_user

FROM silver_cf_tblcustomer p;




Run केल्यावर CF validation (same style)
1) CF source DOB null किती?
%%sql
SELECT COUNT(*) AS cf_dob_null
FROM silver_cf_tblcustomer
WHERE customer_date_of_birth IS NULL;

2) Mock मध्ये CF rows count
%%sql
SELECT COUNT(*) AS cf_rows_in_mock
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

3) 9999 count = DOB null count? (हे main check)
%%sql
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

4) Quick sample
%%sql
SELECT patient_id, src_patient_id, age_range, patient_current_age, patient_gender, patient_gender_code_conformed, z_record_is_active
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
LIMIT 20;

las

5) 
SELECT 
  COUNT(*) total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';


In [ ]:
%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('CF', p.customer_id)                                  AS patient_id,
    CAST(p.customer_id AS STRING)                                AS src_patient_id,

    /* age_range */
    CASE
        WHEN p.customer_date_of_birth IS NULL THEN 'Unknown'
        WHEN TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd') IS NULL THEN 'Unknown'
        WHEN TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd') = DATE '1900-01-01' THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END                                                         AS age_range,

    /* patient_current_age (business safe) */
    CASE
        WHEN p.customer_date_of_birth IS NULL THEN 9999
        WHEN TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd') IS NULL THEN 9999
        WHEN TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd') = DATE '1900-01-01' THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, TO_DATE(p.customer_date_of_birth, 'yyyy-MM-dd'), CURRENT_DATE())
    END                                                         AS patient_current_age,

    p.customer_gender                                            AS patient_gender,

    CASE
        WHEN p.customer_gender IN ('F', 'Female', 'Female (including trans woman)') THEN 'Female'
        WHEN p.customer_gender IN ('M', 'Male', 'Male (including trans man)') THEN 'Male'
        WHEN p.customer_gender IN ('Other', 'Non-Binary', 'Non-binary', 'Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END                                                         AS patient_gender_code_conformed,

    COALESCE(p.customer_status, 0)                               AS z_record_is_active,
    'CF000'                                                      AS z_src_system_instance,
    'CF'                                                         AS z_src_system_id,

    'Unknown'                                                    AS z_src_system_created_date_time,
    'Unknown'                                                    AS z_src_system_created_by_user,
    'Unknown'                                                    AS z_src_system_modified_date_time,
    'Unknown'                                                    AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP()                                          AS z_record_created_date_time,
    CURRENT_USER()                                               AS z_record_created_by_user,
    CURRENT_TIMESTAMP()                                          AS z_record_modified_date_time,
    CURRENT_USER()                                               AS z_record_modified_by_user
FROM silver_cf_tblcustomer p;


In [ ]:
A) CF source count
%%sql
SELECT COUNT(*) AS cf_source_rows
FROM silver_cf_tblcustomer;

B) CF inserted rows in final table
%%sql
SELECT COUNT(*) AS cf_final_rows
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';


Ideally cf_source_rows == cf_final_rows (जर duplicate/filters नाहीत तर).

C) CF: patient_id / src_patient_id null check
%%sql
SELECT
  SUM(CASE WHEN patient_id IS NULL THEN 1 ELSE 0 END)      AS patient_id_null,
  SUM(CASE WHEN src_patient_id IS NULL THEN 1 ELSE 0 END)  AS src_patient_id_null
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

D) CF: DOB nulls in source vs 9999 in final (should match for null DOB + bad placeholder handling)
%%sql
SELECT
  (SELECT COUNT(*) FROM silver_cf_tblcustomer WHERE customer_date_of_birth IS NULL) AS cf_dob_null_in_source,
  (SELECT SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END)
   FROM silver_patient_mock WHERE z_src_system_id='CF')                             AS cf_age_9999_in_final;

3) 🧨 “Bad record” validation (तो 1 record final मध्ये आला का?)

तुझ्या screenshot नुसार bad DOB = 1900-01-01 आणि customer_id = 77408.

A) Source मध्ये तो record confirm
%%sql
SELECT customer_id, customer_date_of_birth
FROM silver_cf_tblcustomer
WHERE customer_id = 77408;

B) Final table मध्ये तो record आला का check (patient_id = CF + customer_id)
%%sql
SELECT
  patient_id,
  src_patient_id,
  age_range,
  patient_current_age,
  z_src_system_id
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND patient_id = 'CF77408';


Expected (आपल्या handling नुसार):

patient_current_age = 9999

age_range = 'Unknown'

C) Final मध्ये 1900-01-01 placeholder किती आहेत? (should be 1 जर फक्त तोच record असेल)
%%sql
SELECT COUNT(*) AS placeholder_1900_count
FROM silver_cf_tblcustomer
WHERE customer_date_of_birth = '1900-01-01';


आणि ते final मध्ये 9999/Unknown म्हणून count होतो का:

%%sql
SELECT COUNT(*) AS placeholder_handled_in_final
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND patient_current_age = 9999;

In [ ]:
1) आधी “bad record” अचूक कोणता आहे ते काढ (CF source मधून)

हे run कर:

%%sql
SELECT
  customer_id,
  customer_date_of_birth
FROM silver_cf_tblcustomer
WHERE customer_date_of_birth IS NOT NULL
  AND COALESCE(
        try_to_timestamp(customer_date_of_birth, 'yyyy-MM-dd HH:mm:ss'),
        try_to_timestamp(customer_date_of_birth, 'yyyy-MM-dd')
      ) IS NULL
LIMIT 50;


यातून जे rows येतील तेच “bad format” आहेत.

2) CF साठी FINAL safe code (bad format handle करून insert होईल)

महत्त्वाचं: ह्यात try_to_timestamp वापरलाय, त्यामुळे bad values मुळे job fail होणार नाही. (bad असेल तर dob_date null होईल आणि age 9999 ठेवतो)

आधी CF delete केलाय म्हणालीस, ठीक आहे. आता हा CF insert code रन कर.

%%sql
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('CF', CAST(p.customer_id AS STRING)) AS patient_id,
    CAST(p.customer_id AS STRING)               AS src_patient_id,

    /* DOB parsed safely (supports both with-time and date-only) */
    CASE
      WHEN dob_date IS NULL THEN 'Unknown'
      WHEN TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE()) < 18 THEN 'Under 18'
      WHEN TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
      WHEN TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
      WHEN TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
      WHEN TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
      WHEN TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
      ELSE '65 and over'
    END AS age_range,

    CASE
      WHEN dob_date IS NULL THEN 9999
      WHEN dob_date = DATE '1900-01-01' THEN 9999
      ELSE TIMESTAMPDIFF(YEAR, dob_date, CURRENT_DATE())
    END AS patient_current_age,

    p.customer_gender AS patient_gender,

    CASE
      WHEN p.customer_gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
      WHEN p.customer_gender IN ('M','Male','Male (including trans man)') THEN 'Male'
      WHEN p.customer_gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
      ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    COALESCE(p.customer_status, 0) AS z_record_is_active,
    'CF000'                         AS z_src_system_instance,
    'CF'                            AS z_src_system_id,
    'Unknown'                       AS z_src_system_created_date_time,
    'Unknown'                       AS z_src_system_created_by_user,
    'Unknown'                       AS z_src_system_modified_date_time,
    'Unknown'                       AS z_src_system_modified_by_user,
    CURRENT_TIMESTAMP()             AS z_record_created_date_time,
    CURRENT_USER()                  AS z_record_created_by_user,
    CURRENT_TIMESTAMP()             AS z_record_modified_date_time,
    CURRENT_USER()                  AS z_record_modified_by_user
FROM (
    SELECT
      p.*,
      TO_DATE(
        COALESCE(
          try_to_timestamp(p.customer_date_of_birth, 'yyyy-MM-dd HH:mm:ss'),
          try_to_timestamp(p.customer_date_of_birth, 'yyyy-MM-dd')
        )
      ) AS dob_date
    FROM silver_cf_tblcustomer p
) p;

3) आता तो “bad record” final table मध्ये आला का ते कसं check करायचं?

तू आधी source मधून bad customer_id काढ (step 1 मधून).

समजा bad customer_id = 77408 (तुझ्या screenshot मध्ये दिसतोय).

(A) Final table मध्ये row आहे का?
%%sql
SELECT *
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND src_patient_id = '77408';

(B) त्या row ला patient_current_age 9999 लागला का?
%%sql
SELECT
  src_patient_id,
  patient_current_age,
  age_range
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND src_patient_id = '77408';


जर parsing fail झाला असेल तर:

patient_current_age = 9999

age_range = 'Unknown' (मी तसंच ठेवलेलं आहे)

last try cf 

In [ ]:
-- 0) (One-time per rerun) delete CF rows from mock table
DELETE FROM silver_patient_mock
WHERE z_src_system_id = 'CF';


-- 1) Insert CF rows (final, business-safe)
INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('CF', CAST(p.customer_id AS STRING))                                  AS patient_id,
    CAST(p.customer_id AS STRING)                                               AS src_patient_id,

    /* age_range (business-safe) */
    CASE
        WHEN dob_dt IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) > 120 THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END                                                                           AS age_range,

    /* patient_current_age (business-safe) */
    CASE
        WHEN dob_dt IS NULL THEN 9999
        WHEN TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE()) > 120 THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, dob_dt, CURRENT_DATE())
    END                                                                           AS patient_current_age,

    p.customer_gender                                                             AS patient_gender,

    CASE
        WHEN p.customer_gender IN ('F','Female','Female (including trans woman)') THEN 'Female'
        WHEN p.customer_gender IN ('M','Male','Male (including trans man)')       THEN 'Male'
        WHEN p.customer_gender IN ('Other','Non-Binary','Non-binary','Other gender not listed') THEN 'Other'
        ELSE 'Unknown'
    END                                                                           AS patient_gender_code_conformed,

    COALESCE(p.customer_status, 0)                                                AS z_record_is_active,
    'CF000'                                                                       AS z_src_system_instance,
    'CF'                                                                          AS z_src_system_id,

    'Unknown'                                                                     AS z_src_system_created_date_time,
    'Unknown'                                                                     AS z_src_system_created_by_user,
    'Unknown'                                                                     AS z_src_system_modified_date_time,
    'Unknown'                                                                     AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP()                                                           AS z_record_created_date_time,
    CURRENT_USER()                                                                AS z_record_created_by_user,
    CURRENT_TIMESTAMP()                                                           AS z_record_modified_date_time,
    CURRENT_USER()                                                                AS z_record_modified_by_user
FROM (
    SELECT
        p.*,

        /* Safe DOB parsing -> DATE */
        TO_DATE(
            COALESCE(
                try_to_timestamp(NULLIF(TRIM(p.customer_date_of_birth), ''), 'yyyy-MM-dd HH:mm:ss'),
                try_to_timestamp(NULLIF(TRIM(p.customer_date_of_birth), ''), 'yyyy-MM-dd')
            )
        ) AS dob_dt

    FROM silver_cf_tblcustomer p
) p;


In [ ]:
-- A) CF rows inserted count
SELECT COUNT(*) AS cf_rows
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

-- B) How many got 9999 (invalid / missing / crazy DOB)
SELECT
  COUNT(*) AS total,
  SUM(CASE WHEN patient_current_age = 9999 THEN 1 ELSE 0 END) AS age_9999
FROM silver_patient_mock
WHERE z_src_system_id = 'CF';

-- C) Show the records that became 9999 (to inspect)
SELECT src_patient_id, patient_id, patient_current_age, age_range
FROM silver_patient_mock
WHERE z_src_system_id = 'CF'
  AND patient_current_age = 9999
LIMIT 50;


TM£

In [ ]:
-- Rerun safe
DELETE FROM silver_patient_mock
WHERE z_src_system_id = 'TM3';

INSERT INTO TABLE silver_patient_mock
SELECT
    CONCAT('TM3', p.client_reference, p.client_pk) AS patient_id,
    CAST(p.client_pk AS STRING)                    AS src_patient_id,

    /* age_range */
    CASE
        WHEN p.client_date_of_birth IS NULL THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) > 120 THEN 'Unknown'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) < 18 THEN 'Under 18'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 18 AND 24 THEN '18-24'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 25 AND 34 THEN '25-34'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 35 AND 44 THEN '35-44'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 45 AND 54 THEN '45-54'
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) BETWEEN 55 AND 64 THEN '55-64'
        ELSE '65 and over'
    END AS age_range,

    /* patient_current_age */
    CASE
        WHEN p.client_date_of_birth IS NULL THEN 9999
        WHEN TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE()) > 120 THEN 9999
        ELSE TIMESTAMPDIFF(YEAR, p.client_date_of_birth, CURRENT_DATE())
    END AS patient_current_age,

    p.client_gender AS patient_gender,

    CASE
        WHEN UPPER(TRIM(p.client_gender)) IN ('F','FEMALE','FEMALE (INCLUDING TRANS WOMAN)') THEN 'Female'
        WHEN UPPER(TRIM(p.client_gender)) IN ('M','MALE','MALE (INCLUDING TRANS MAN)') THEN 'Male'
        WHEN UPPER(TRIM(p.client_gender)) IN ('OTHER','NON-BINARY','OTHER GENDER NOT LISTED') THEN 'Other'
        ELSE 'Unknown'
    END AS patient_gender_code_conformed,

    1 AS z_record_is_active,
    'TM3000' AS z_src_system_instance,
    'TM3' AS z_src_system_id,

    'Unknown' AS z_src_system_created_date_time,
    'Unknown' AS z_src_system_created_by_user,
    'Unknown' AS z_src_system_modified_date_time,
    'Unknown' AS z_src_system_modified_by_user,

    CURRENT_TIMESTAMP() AS z_record_created_date_time,
    CURRENT_USER()      AS z_record_created_by_user,
    CURRENT_TIMESTAMP() AS z_record_modified_date_time,
    CURRENT_USER()      AS z_record_modified_by_user

FROM silver_tm3_dim_clients p;


In [ ]:
-- Row count match
SELECT COUNT(*) FROM silver_tm3_dim_clients;

SELECT COUNT(*)
FROM silver_patient_mock
WHERE z_src_system_id = 'TM3';


-- Null age check
SELECT COUNT(*)
FROM silver_patient_mock
WHERE z_src_system_id = 'TM3'
AND patient_current_age IS NULL;


-- 9999 count
SELECT COUNT(*)
FROM silver_patient_mock
WHERE z_src_system_id = 'TM3'
AND patient_current_age = 9999;
